In [9]:
from pymmcore_camera_sim import SimCameraDevice
from pymmcore_slm_sim import SimSLMDevice
from pymmcore_stage_sim import SimStageDevice
from pymmcore_state_device_sim import SimStateDevice
from pymmcore_shutter_sim import SimShutterDevice
import napari
from pymmcore_plus.experimental.unicore import UniMMCore
from microscope_sim import MicroscopeSim
from pymmcore_plus import CMMCorePlus, Keyword
from pathlib import Path

In [10]:
core = UniMMCore()

microscope_simulation = MicroscopeSim()
print(core.getLoadedDevices())
#-----------------------------------------
# unload all devices
core.unloadAllDevices()

print(core.getLoadedDevices())
#-----------------------------------------------------
# load device
core.loadPyDevice("Camera", SimCameraDevice(core=core, microscope_sim=microscope_simulation))
core.loadPyDevice("XYStage", SimStageDevice(microscope_sim=microscope_simulation))
core.loadPyDevice("LED", SimStateDevice(label="LED",state_dict={0:"UV", 1:"BLUE", 2:"CYAN", 3:"GREEN", 4:"YELLOW", 5:"ORANGE", 6:"RED"}, microscope_sim=microscope_simulation))
core.loadPyDevice("Filter Wheel", SimStateDevice(label="Filter Wheel",state_dict={0:"Electra1(402/454)", 1:"SCFP2(434/474)", 2:"TagGFP2(483/506)", 3:"obeYFP(514/528)", 5:"mRFP1-Q667(549/570)", 6:"mScarlet3(569/582)", 7:"miRFP670(642/670)"}, microscope_sim=microscope_simulation))
core.loadPyDevice("Shutter", SimShutterDevice())
#------------------------------------
print(core.getLoadedDevices())
# initialize device
#core.initializeDevice("Camera")
#core.initializeDevice("XYStage")
#core.initializeDevice("Filter Wheel")
#core.initializeDevice("LED")
#core.initializeDevice("Shutter")

core.initializeAllDevices()

print(core.getLoadedDevices())

print(core.getDeviceInitializationState("LED"))

#---------------------------
# Set initial value of some device
core.setCameraDevice("Camera")
core.setXYStageDevice("XYStage")
core.setShutterDevice("Shutter")
core.setState("LED", 0)
core.setState("Filter Wheel", 0)

core.defineConfigGroup("Fake")
core.defineConfigGroup("Real")
core.defineConfig("Fake", "nucleus-channel", "LED", "Label", "ORANGE")
core.defineConfig("Fake", "nucleus-channel", "Filter Wheel", "Label", "mScarlet3(569/582)")
core.defineConfig("Real", "membrane-channel", "LED", "Label", "RED")
core.defineConfig("Real", "membrane-channel", "Filter Wheel", "Label", "miRFP670(642/670)")

core.setConfig("Fake", "nucleus-channel")

#print(core.getDeviceInitializationState("LED").__dict__)
#viewer = napari.Viewer()

#viewer.window.add_plugin_dock_widget(plugin_name='napari-micromanager')

#napari.run()

('Core',)
('Core',)
('Camera', 'XYStage', 'LED', 'Filter Wheel', 'Shutter', 'Core')
('Camera', 'XYStage', 'LED', 'Filter Wheel', 'Shutter', 'Core')
1


RuntimeError: No device with label "LED"

In [11]:
core.getAllowedPropertyValues("LED", "Label")

('UV', 'BLUE', 'CYAN', 'GREEN', 'YELLOW', 'ORANGE', 'RED')

## Simplified

In [4]:
core.getPropertyType("Camera", "brightness")

float

In [8]:
core.reset()
core = UniMMCore()# restaret the kernel so the singleton reset or just use core.reset()
microscope_simulation = MicroscopeSim()

# load device python
core.loadPyDevice("Camera", SimCameraDevice(core=core, microscope_sim=microscope_simulation))
core.loadPyDevice("XYStage", SimStageDevice(microscope_sim=microscope_simulation))
core.initializeAllDevices()

# initialize device
core.setCameraDevice("Camera")
core.setXYStageDevice("XYStage")

# configure new group
core.defineConfigGroup("test")
# define new configuration
core.defineConfig("test","state-0","Camera","brightness",25.0)
core.defineConfig("test","state-0","Camera",Keyword.Exposure,2.0)

core.setConfig("test", "state-0")

TypeError: Wrong number or type of arguments for overloaded function 'CMMCore_defineConfig'.
  Possible C/C++ prototypes are:
    CMMCore::defineConfig(char const *,char const *)
    CMMCore::defineConfig(char const *,char const *,char const *,char const *,char const *)


### Try to override UniMMCore class

In [ ]:
from registry import DEVICE_REGISTRY
from pathlib import Path
from pymmcore_plus.experimental.unicore import UniMMCore
from microscope_sim import MicroscopeSim
from pymmcore_plus import CMMCorePlus, Keyword
import devices


class MyUniMMCore(UniMMCore): # create subclass for the my current example, later if it does work add to the implementation

    def __init__(self, microscope_sim: MicroscopeSim | None = None):
        super().__init__()
        self.microscope_sim = microscope_sim or MicroscopeSim()

    # def getConfigState(
    #     self, group: str, config: str, *, native: bool = False
    # ):
    #     print(self.getAvailableConfigGroups())
    #     print(self.getAvailableConfigs(group))
    #     if group in self.getAvailableConfigGroups() and config in self.getAvailableConfigs(group):
    #         return self.getConfigData(group, config,native=native)
    #     else:
    #         return ()

    def loadSystemConfiguration(
        self, fileName: str | Path = "test_config.cfg"
    ) -> None:
        # COPY FROM PYMMCORE-PLUS for the moment
        fpath = Path(fileName).expanduser()
        if not fpath.exists() and not fpath.is_absolute() and self._mm_path:
            fpath = Path(self._mm_path) / fileName
        if not fpath.exists():
            raise FileNotFoundError(f"Path does not exist: {fpath}")
        self._last_sys_config = str(fpath.resolve())
        # open file and read lines
        with open(self._last_sys_config, "r") as config_file:
            list_of_lines = config_file.readlines() # read lines and saved in a list
        config_file.close() # close the file

        for line in list_of_lines: # iterate over the list of lines
            print(line)
            # Get through the list
            if line.startswith("#"):
                continue
            elif line.startswith("Device"):
                # load the device in the system if not present
                # Device,Name of the device, Python device or C++, device label
                _, device_label,module_name,device_name = line.strip().split(',')
                try:
                    if device_label in self.getDeviceAdapterNames():
                        self.loadDevice(device_label,module_name, device_name)
                    elif device_name in DEVICE_REGISTRY.keys(): # checks if always works with python
                        fact = DEVICE_REGISTRY[device_name]
                        # Try to find better implementation
                        if device_label == "Camera":
                            pydev = fact(self, self.microscope_sim)
                        elif device_label == "XYStage":
                            pydev = fact(self.microscope_sim)
                        elif device_label == "Shutter":
                            pydev = fact()
                        elif device_label == "LED":
                            pydev = fact(device_label,{0:""},self.microscope_sim)
                            #{0:"UV", 1:"BLUE", 2:"CYAN", 3:"GREEN", 4:"YELLOW", 5:"ORANGE", 6:"RED"}
                        else:
                            pydev = fact(device_label,{0:""},self.microscope_sim)
                            #{0:"Electra1(402/454)", 1:"SCFP2(434/474)", 2:"TagGFP2(483/506)", 3:"obeYFP(514/528)", 4:"mRFP1-Q667(549/570)", 5:"mScarlet3(569/582)", 6:"miRFP670(642/670)"}
                        self.loadPyDevice(device_label, pydev)
                    else:
                        raise RuntimeError(f"Unknown device or adapter for {module_name}")
                except Exception as e:
                    # if module_name not in self.getDeviceAdapterNames():
                    #     # is a python device
                    #     pydev = self._get_py_device_instance(module_name, device_name)
                    #     self.loadPyDevice(device_label, pydev)
                    raise RuntimeError(f"Failed to load {device_label}: {e}")
                # initialize the device
                self.initializeDevice(device_label)
                print(self.getLoadedDevices())

                # set initial value
                if 'Camera' in device_name:
                    self.setCameraDevice(device_label)
                elif 'Stage' in device_name:
                    self.setXYStageDevice(device_label)
                elif 'Shutter' in device_name:
                    self.setShutterDevice(device_label)
                elif 'State' in device_name:
                    self.setState(device_label,0) # by default always has default value of 0

                # think if something is missing
            elif line.startswith("Label"):
                # load the state device here
                _, device_label,state_int, state_label = line.strip().split(',')
                # check if device is already loaded
                if device_label in self.getLoadedDevices(): # device is loaded
                    self.defineStateLabel(device_label, int(state_int), state_label)
                else: # device is not present. I assume in this case is a StateDevice, because it follows the mapping implementation
                    continue # for the moment leave like this, don't know yet if its important this condition
            elif line.startswith("ConfigGroup"):
                # load and sets differents configuration
                _, configuration_group, configuration_presets, device_label, property_name, property_value = line.strip().split(',')

                if not self.isGroupDefined(configuration_group): # check if the configuration group exists
                    self.defineConfigGroup(configuration_group)
                if device_label not in self.getLoadedDevices():
                    raise RuntimeError(f"Failed to load {device_label}: {device_label} is not a loaded device.")
                print(self.getDevicePropertyNames(device_label))
                print(property_name)
                if self.hasProperty(device_label, property_name):# the property is not present in the device settings
                    if not self.isPropertyReadOnly(device_label, property_name): # property is not readonly
                        if property_value in self.getAllowedPropertyValues(device_label, property_name):# checks if the value of the property is in the list of allowed value
                            self.defineConfig(configuration_group, configuration_presets, device_label, property_name, property_value) # define new configuration/settings
                        else:
                            raise RuntimeError(f"Failed to define new configuration: {property_value} is not a valid value for property {property_name}")
                    else:
                        raise RuntimeError(f"Failed to define new configuration: {property_value} is a ReadOnly property")
                else:
                    raise RuntimeError(f"Failed to define new configuration: {property_value} is not a property of the device {device_label}")

            # in the demo there is also the pixelsizeconfig. For the moment i dont add it, its not important for my example

In [ ]:
new_core = MyUniMMCore()

In [ ]:
new_core.loadSystemConfiguration()